In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd
from bpemb import BPEmb

from fastai.callbacks import *
from fastai.datasets import *
from fastai.imports import nn, torch
from fastai.metrics import *
from fastai.text import *
from fastai.text.data import TextLMDataBunch
from fastai.text.transform import BaseTokenizer, Tokenizer, Vocab
from fastai.train import *
import fastai

import news_utils.fastai
torch.cuda.set_device(1)

In [2]:
fastai.__version__

'1.0.42'

In [3]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

In [4]:
itos = dict(enumerate(bpemb_de.words))
itos[0] = 'xxunk'
itos[1] = 'xxpad'
voc = Vocab(itos)

In [86]:
fold = '1'
cat = 'SentimentNegative'
cat = 'PersonalStories'
# model_id = '2019_ 2_20_12_23_12_684867'
model_id = '2019_ 3_25_17_00_53_007523'
exp_path = '/mnt/data/group07/johannes/ompc/exp/' + cat + '_' + fold

In [6]:
data_lm = TextLMDataBunch.load(Path('/mnt/data/group07/johannes/germanlm/exp_1'))

In [60]:
learn_lm = language_model_learner(data_lm, drop_mult=0).load(Path('/mnt/data/group07/johannes/germanlm/exp_1/models/' + model_id, device="cpu"))

In [136]:
df_all = pd.read_pickle('/mnt/data/group07/johannes/ompc/unla.pkl')
df_all = df_all.dropna(subset=['Body'])

In [27]:
df_all['text_ids'] = df_all['Body'].apply(lambda x: bpemb_de.encode_ids(x))

In [38]:
df_all.shape

(950926, 11)

In [140]:
df_all_train = df_all[df_all['ID_Article'] < 11500]
df_all_val = df_all[df_all['ID_Article'] >= 11500]

In [141]:
df_all_train.shape

(902726, 10)

In [142]:
df_all_val.shape

(48200, 10)

In [52]:
data_lm_ft = TextLMDataBunch.from_ids(bs=128, path='/mnt/data/group07/johannes/ompc/exp/unlab',vocab=data_lm.vocab, train_ids=df_all_train['text_ids'], valid_ids=df_all_val['text_ids'])

In [61]:
learn_lm.data = data_lm_ft

In [62]:
learn_lm.unfreeze()

In [57]:
optim_lr = news_utils.fastai.get_optimal_lr(learn_lm, runs=1)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.
best lr: 0.0022908676527677745


In [58]:
optim_lr

0.0022908676527677745

In [63]:
learn_lm.fit_one_cycle(1, 0.002)

epoch,train_loss,valid_loss,accuracy
1,4.124428,4.141690,0.289924


In [64]:
learn_lm.save_encoder('enc1')

In [134]:
learn_lm

NameError: name 'learn_lm' is not defined

In [132]:
shutil.copy('/mnt/data/group07/johannes/ompc/exp/unlab/models/enc1.pth', '/mnt/data/group07/johannes/ompc/exp/' + cat + '_' + fold +'/models/enc1.pth')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/group07/johannes/ompc/exp/unlab/models/enc1.pth'

In [55]:
shutil.move('/mnt/data/group07/johannes/germanlm/exp_1/models/enc1.pth', '/mnt/data/group07/johannes/ompc/exp/' + cat + '_' + fold +'/models/enc1.pth')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/group07/johannes/germanlm/exp_1/models/enc1.pth'

In [91]:
train_df = pd.read_pickle(Path('/mnt/data/group07/johannes/ompc/data_ann')/cat/fold/'train.pkl')
test_df = pd.read_pickle(Path('/mnt/data/group07/johannes/ompc/data_ann')/cat/fold/'test.pkl')

In [92]:
train_df = train_df.dropna(subset=['Body'])
test_df = test_df.dropna(subset=['Body'])

In [93]:
train_df['text_ids'] = train_df['Body'].apply(lambda x: bpemb_de.encode_ids(x))
test_df['text_ids'] = test_df['Body'].apply(lambda x: bpemb_de.encode_ids(x))

In [110]:
train_df

,ID_Post,Category,Value,Fold,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_Article,text_ids
13,3326,PersonalStories,0,3,3257.0,13,18083,2015-06-26 14:20:56.203,online,None,Top qualifizierte Leute verdienen auch viel.,1,3,Kiaroom/Karriere/Karriereratgeber/Arbeitrecht/...,2015-03-26 12:57:58.00,"Konkurrenzklausel: Wer zahlen muss, wenn man a...","<div class=""section"" id=""content-main"" itempro...","[3795, 14696, 11709, 2316, 4699, 200, 1049, 24..."
117,5321,PersonalStories,0,3,5316.0,66,7054,2015-06-01 21:15:03.110,online,None,"Gott sei dank ist für sie eine Umfrage alles, ...",2,1,Newsroom/Inland/bundeslaender/ipowien,2015-06-01 17:19:56.00,Wiener SPÖ will FPÖ nicht in die Hände spielen,"<div class=""section"" id=""content-main"" itempro...","[2606, 869, 6645, 142, 137, 178, 149, 1839, 18..."
260,5590,PersonalStories,0,3,NaN,76,25979,2015-06-01 23:14:56.480,online,""" Die FPÖ wird aus allen Rohren schießen und d...","Sorry, aber die FPÖ tut eigentlich gar nichts ...",7,1,Meinung/Kommentare/Voelker,2015-06-01 17:49:17.00,Koalitionäre Handlanger der FPÖ,"<div class=""section"" id=""content-main"" itempro...","[3321, 1780, 24940, 499, 31, 21750, 14803, 609..."
301,6015,PersonalStories,0,3,6014.0,94,28638,2015-06-02 14:21:36.867,online,None,Weil es dein meisten Leuten verständlicherweis...,2,0,Newsroom/Web/Games/Berufsspieler,2015-06-02 11:11:03.00,"Sexistischer Shitstorm gegen ""Fifa 16"": Ein Ei...","<div class=""section"" id=""content-main"" itempro...","[2173, 234, 13740, 2159, 23776, 14666, 7424, 1..."
322,8213,PersonalStories,0,3,NaN,113,21711,2015-06-02 20:24:58.513,online,None,Na wer weis was da vorgefallen ist...,0,0,Newsroom/Sport/Fussball/FIFA,2015-06-02 18:52:48.00,"Blatter geht, der Fußball bleibt","<div class=""section"" id=""content-main"" itempro...","[4892, 215, 10604, 607, 326, 190, 20579, 142, ..."
354,9724,PersonalStories,0,3,9721.0,140,9920,2015-06-03 12:20:29.250,online,None,Gabalier?\r\nKünstler?\r\nBruahahaha!,5,1,Meinung/Kommentare/Sulzbacher,2015-06-03 09:47:45.00,Festplattenabgabe: Gabalier bekommt nun Geld f...,"<div class=""section"" id=""content-main"" itempro...","[1078, 22647, 24974, 0, 6578, 24974, 0, 867, 1..."
374,12986,PersonalStories,0,3,NaN,161,7273,2015-06-08 16:28:25.143,online,"Juhu, ich werde reich! :)","Very Urgent Bastard,\r\n\r\nl am Mr. Benson Ba...",2,0,Newsroom/User/Userforum,2015-06-03 22:09:52.00,Off-Topic-Forum #56,"<div class=""section"" id=""content-main"" itempro...","[81, 24950, 654, 1804, 10808, 346, 24940, 0, 2..."
419,13038,PersonalStories,0,3,13036.0,161,20571,2015-06-18 17:55:57.577,online,None,was die meise gibt kein black metal konzert?,3,0,Newsroom/User/Userforum,2015-06-03 22:09:52.00,Off-Topic-Forum #56,"<div class=""section"" id=""content-main"" itempro...","[607, 31, 1543, 132, 1082, 1502, 5244, 9890, 5..."
446,13060,PersonalStories,0,3,NaN,161,18567,2015-06-22 12:09:32.440,online,None,"Musst nicht, sonst gäbs weniger Spaß! ^^",1,0,Newsroom/User/Userforum,2015-06-03 22:09:52.00,Off-Topic-Forum #56,"<div class=""section"" id=""content-main"" itempro...","[434, 34, 275, 24940, 5202, 27, 24942, 3496, 2..."
481,13076,PersonalStories,0,3,NaN,164,12465,2015-06-04 07:44:08.280,online,Es sind immer dieselben bigotten verlogenen Be...,die mit ihren eigenen sexuellen - mitunter kra...,4,5,Newsroom/International/Europa/deutschland,2015-06-03 18:27:52.00,Homophobie: Empörung über Regierungschefin des...,"<div class=""section"" id=""content-main"" itempro...","[31, 90, 1302, 2367, 22099, 570, 13889, 3997, ..."


In [125]:
test_df = test_df.iloc[:900]

In [126]:
data = TextClasDataBunch.from_ids(bs=65,pad_first=False,path=exp_path, vocab=data_lm.vocab, classes=[0, 1], train_lbls=train_df['Value'], valid_lbls=test_df['Value'], train_ids=train_df['text_ids'], valid_ids=test_df['text_ids'])

In [127]:
data.save()

In [130]:
learn = text_classifier_learner(data, drop_mult=0.8)
learn.load_encoder('enc1')

hidden:  0.24
input:  0.32000000000000006
embed:  0.04000000000000001
weight:  0.4


In [131]:
lr = 0.001
lrs = [lr / (2.6 ** (4 - x)) for x in range(4)] + [lr]
# learn.metrics += [KappaScore()]

learn.metrics += [KappaScore(), news_utils.fastai.F1Bin(), news_utils.fastai.PrecBin(), news_utils.fastai.RecaBin()]

#         learn.callbacks += [
#             SaveModelCallback(learn, name=exp_id, monitor='kappa_score'),
#             news_utils.fastai.SacredLogger(learn, ex),
#         ]

for i in range(1, 4):
    epochs = 2
    if i in [1, 2]:
        learn.freeze_to(-i)
    else:
        learn.unfreeze()
        epochs = 200
    learn.fit(epochs, np.array(lrs))

epoch,train_loss,valid_loss,accuracy,kappa_score,F1_bin,prec_bin,reca_bin
1,0.397289,0.396970,0.818889,0.264691,0.360784,0.282209,0.500000
2,0.378564,0.488013,0.744444,0.117354,0.272152,0.263804,0.281046


epoch,train_loss,valid_loss,accuracy,kappa_score,F1_bin,prec_bin,reca_bin
1,0.335557,0.497215,0.744444,0.067122,0.217687,0.196319,0.244275
2,0.317733,0.432817,0.790000,0.250859,0.376238,0.349693,0.407143


epoch,train_loss,valid_loss,accuracy,kappa_score,F1_bin,prec_bin,reca_bin
1,0.335028,0.466237,0.758889,0.079364,0.216606,0.184049,0.263158
2,0.295646,0.463658,0.761111,0.134677,0.276094,0.251534,0.305970
3,0.290698,0.465089,0.774444,0.195367,0.330033,0.306748,0.357143
4,0.283977,0.420658,0.785556,0.235004,0.363036,0.337423,0.392857
5,0.297298,0.463195,0.771111,0.242814,0.383234,0.392638,0.374269
6,0.287708,0.429223,0.782222,0.209118,0.337838,0.306748,0.375940
7,0.271759,0.445188,0.760000,0.270998,0.419355,0.478528,0.373206
8,0.244242,0.388457,0.818889,0.174433,0.248848,0.165644,0.500000
9,0.254558,0.428941,0.780000,0.295346,0.431034,0.460123,0.405405
10,0.255612,0.441373,0.767778,0.264545,0.407932,0.441718,0.378947


KeyboardInterrupt: 